In [ ]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
from splinter import Browser

In [ ]:
#!which chromedriver

In [ ]:
# Initialize Chromedriver 
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless= False)

In [ ]:
########################### NASA MARS NEWS ###########################

In [ ]:
# URL for scrape
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
type(soup)

In [ ]:
# Examine the results, then determine element that contains news title and news paragraph text
print(soup.prettify())


In [ ]:
# Find the content_title class, which contains the titles of the news articles
content_title = soup.find(class_="content_title")
print(content_title)

In [ ]:
news_title = content_title.find("a").get_text()
print(news_title)

In [ ]:
news_p = soup.find(class_="rollover_description_inner").get_text()
print(news_p)

In [4]:
########################### JPL MARS SPACE IMAGES - FEATURED IMAGE ###########################
# URL for scrape
url = "https://www.jpl.nasa.gov/spaceimages/"
browser.visit(url)

In [5]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

<Response [200]>


In [16]:
# Create BeautifulSoup object; parse 
soup = BeautifulSoup(response.text, 'lxml')
type(soup)

bs4.BeautifulSoup

In [17]:
# Examine the results, then determine element that contains images and url
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html>
 <!-- START HEADER: "DEFAULT" -->
 <head>
  <meta charset="utf-8"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <title>
   Space Images
  </title>
  <link href="/assets/stylesheets/manifest.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="/assets/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>
  <script src="/assets/javascripts/public_manifest.js" type="text/javascript">
  </script>
  <script src="/assets/javascripts/vendor/jquery.fancybox.js" type="text/javascript">
  </script>
  <script src="/assets/javascripts/vendor/jquery.fancybox-thumbs.js" type="text/javascript">
  </script>
 </he

In [18]:
# Find image url to full size
full_size = soup.find(class_ ="carousel_item")
print(full_size)

<article alt="Galaxy in Different Lights" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA18840-1920x1200.jpg');">
<div class="default floating_text_area ms-layer">
<h2 class="category_title">
</h2>
<h2 class="brand_title">
				  FEATURED IMAGE
				</h2>
<h1 class="media_feature_title">
				  Galaxy in Different Lights				</h1>
<div class="description">
</div>
<footer>
<a class="button fancybox" data-description="The comparison from NASA's Hubble telescope and Chandra X-ray Observatory highlights how different the universe can look when viewed in other wavelengths of light. M82 is located 12 million light-years away in the Ursa Major constellation." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA18840_ip.jpg" data-link="/spaceimages/details.php?id=PIA18840" data-title="Galaxy in Different Lights" id="full_image">
					FULL IMAGE
				  </a>
</footer>
</div>
<div class="gradient_container_top"></div>
<div class="gra

In [25]:
# this funciton will return a dictionary for image that we can use to store in mongo DB
def img_portfolio(img_src,alt):
    return {
        "src": img_src,
        "alt": alt
    }

In [33]:
# List comprehension to find only the image
#url_string =  [img["src"] for img in soup.find_all("img") if "wallpaper" in img["src"]]
#print(url_string)

all_images = []
all_Images = soup.find_all('img')
for img in all_Images:
    if(img.attrs.get('srcset')):
        _altInfo = img.attrs.get('alt')
                #will give an array of two src
        sources = img.attrs.get('srcset').split(',')
                # we need a smalest source of it                  
        smal_img_src = sources[0].split(' ')[0]
                # and we get our picture info in pretty dictionary                 
                # print(img_portfolio(smal_img_src,_altInfo))
                # Now we can append our data to array and return it 
        all_images.append(img_portfolio(smal_img_src,_altInfo))
        data.append(img_portfolio(smal_img_src,_altInfo))
        print(all_images)    


In [29]:
data = scrape()
pd.DataFrame(data)

""


In [ ]:
########################### MARS WEATHER ###########################
# Visit mars weather twitter account and scrape the latest mars weather tweet from the page
url = "http://twitter.com/marswxreport?lang=en"
# Save tweet text for the report as variable
browser.visit(url)

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
type(soup)

In [ ]:
# Examine the results, then determine element that contains mars weather 
print(soup.prettify())

In [ ]:
mars_weather = soup.find(class_="js-tweet-text-container").get_text()
print(mars_weather)

In [ ]:
########################### MARS FACTS ###########################
# Visit the mars facts webpage, use pandas to scrape the table containing facts about the planet including diameter, mass, etc
facts_url = "http://space-facts.com/mars"

tables = pd.read_html(facts_url)
tables

In [ ]:
# Convert list to dataframe
facts_df = tables[0]

# Set first column to index
facts_df.set_index(0, inplace=True)
facts_df.head(8)

In [ ]:
# Use pandas to convert the data to a HTML table string
facts_table_html = facts_df.to_html(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, sparsify=None, index_names=True, justify=None, bold_rows=True, classes=None, escape=True, max_rows=None, max_cols=None, show_dimensions=False, notebook=False, decimal='.', border=None)
print(facts_table_html)

In [ ]:
########################### MARS HEMISPHERES ###########################

In [ ]:
# Visit the USGS Astrogeology site to obtain hi-res images for each of Mars' hemispheres
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')
type(soup)

In [ ]:
# Examine the results, then determine element that contains images
print(soup.prettify())

In [ ]:
# Find all image tags
images = soup.find_all("img")
print(images)
#type(images)

In [ ]:
# List comprehension to find only the hemispheres images
img_url = [img["src"] for img in soup.find_all("img") if "Hemisphere" in img["alt"]]
print(img_url)

In [ ]:
# List comprehension to find only the hemisphere names
hemisphere_title = [img["alt"] for img in soup.find_all("img") if "Hemisphere" in img["alt"]]
print(hemisphere_title)

In [ ]:
# Use a python dictionary to store the data using keys img_url and hemisphere title
hemisphere_dictionary = dict(zip(keys, values))

def hemisphere_dict(img_url, hemisphere_title):
    return {
        "": img_url,
        "": title
    }


# Append the dictionary with the image url string and the hemisphere title to a list; the list will contain one dictionary for each hemisphere
